In [31]:
import chromadb
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import WikipediaLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
# load gemini api key form .env file
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-pro" , api_key=api_key)

In [ ]:
embedding_function = HuggingFaceBgeEmbeddings(
    model_name = "BAAI/bge-large-en-v1.5",
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

In [22]:
chunk_size = 600
chunk_overlap = 100

chat = llm

In [53]:
# loader = WikipediaLoader(query="Pneumonia", load_max_docs=5)
# documents = loader.load()
# documents
docs = [TextLoader("DB/scraped_faqs.txt").load()]

In [57]:
documents = docs[0]
documents

[Document(metadata={'source': 'DB/scraped_faqs.txt'}, page_content='Question: Airport facilities related queries:\nAnswer: Is Wi-Fi facility available at the airport?The airport is Wi-Fi enabled and passengers can enjoy free Wi-Fi while they are at the airport. A free\nwifi link !!Free_CAAN_WorldLink_WI-Fi!! is available to connect with internet facility.Where do I contact for any query and flight information at the airport?You can contact TIA help desk or Police information desk at the canopy area, Information desk at the check in area, help desk at the departure area, TIA information desk or immigration help desk at the arrival area, Customs help desk at the customs area, Nepal Airlines help desk and Tourist police information center for query or any information about flight at the airport.Is wheelchair assistance available?Yes, Wheelchairs are available at the airport. Please contact your airline for further support. You may contact help desk for the assistance.Is there any business

In [60]:
# text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
docs = text_splitter.split_documents(documents=documents)


In [61]:
db = Chroma.from_documents(docs, embedding_function, persist_directory="output/Airpot.db")

In [62]:
from langchain.retrievers.multi_query import MultiQueryRetriever
mq_retriever = MultiQueryRetriever.from_llm(retriever = db.as_retriever(), llm = chat)

In [68]:
query = "What kind of visa is  Gratis Visa?"
retrieved_docs = mq_retriever.get_relevant_documents(query=query)
retrieved_docs

[Document(metadata={'source': 'DB/scraped_faqs.txt'}, page_content="receipt with barcode, please print it out and bring it along for acquiring visa. It works for fifteen days and becomes invalid then after. If so, you will have to fill it up again.• Step 2:>Make payment at the bank according to your visa requirement ( 15/30/90 Days).>Get the receipt.>Proceed to the appropriate immigration desk and hand in your documents to an immigration officer for visa processing.You can also visit the link for more information: https://www.immigration.gov.np/page/visa-on-arrivalWhat are the different kinds of “On Arrival Visa”?All 'On Arrival Visas' are Tourist Visas. In"),
 Document(metadata={'source': 'DB/scraped_faqs.txt'}, page_content='have children of 5 years and 6 years. Does he/she need a Visa?Yes, however, they get it for free. Minors below 10 years get Gratis visa.What is Gratis Visa?It is a kind of Visa that is issued for free.I am an Indian citizen? I am travelling to Nepal. What documen

In [69]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

retriever = db.as_retriever()
# chat = ChatOpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(chat)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)
compressed_docs = compression_retriever.get_relevant_documents(query = query)
compressed_docs

[Document(metadata={'source': 'DB/scraped_faqs.txt'}, page_content='It is a kind of Visa that is issued for free.'),
 Document(metadata={'source': 'DB/scraped_faqs.txt'}, page_content='Minors below 10 years get Gratis visa.')]

In [72]:
from langchain.retrievers.document_compressors import LLMChainFilter

compressor = LLMChainFilter.from_llm(chat)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)
compressed_docs = compression_retriever.get_relevant_documents(query = query)
compressed_docs

[Document(metadata={'source': 'DB/scraped_faqs.txt'}, page_content='================================================================================\n\nQuestion: I have children of 5 years and 6 years. Does he/she need a Visa?\nAnswer: Yes, however, they get it for free. Minors below 10 years get Gratis visa.\n\n================================================================================\n\nQuestion: What is Gratis Visa?\nAnswer: It is a kind of Visa that is issued for free.\n\n================================================================================'),
 Document(metadata={'source': 'DB/scraped_faqs.txt'}, page_content='================================================================================\n\nQuestion: I am an NRN card holder. How many days Gratis visa do I get?\nAnswer: Gratis visa to an NRN is issued for 30 days at the moment. It is subject to change as per directive issued by Department of Immigration and Ministry of Home Affairs.\n\n============================

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

# using similarity threshold of 0.6
embeddings_filter  = EmbeddingsFilter(embeddings=embedding_function, similarity_threshold=0.6)
compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)
compressed_docs = compression_retriever.get_relevant_documents(query = query)
print(compressed_docs)


[_DocumentWithState(metadata={'source': 'DB/scraped_faqs.txt'}, page_content='================================================================================\n\nQuestion: I have children of 5 years and 6 years. Does he/she need a Visa?\nAnswer: Yes, however, they get it for free. Minors below 10 years get Gratis visa.\n\n================================================================================\n\nQuestion: What is Gratis Visa?\nAnswer: It is a kind of Visa that is issued for free.\n\n================================================================================', state={'embedded_doc': [0.04241647943854332, 0.017436906695365906, -0.014694740995764732, 0.04598243534564972, -0.014572450891137123, -0.0447155199944973, 0.010975123383104801, 0.03429734706878662, -0.0196666456758976, 0.02898671105504036, 0.016477389261126518, 0.0063203866593539715, -0.011835022829473019, -0.00011018172517651692, -0.04586455971002579, -0.0016232904745265841, -0.035794660449028015, 0.0051098265685141